In [52]:
#Import
from spotify import Client, OAuth
import pandas as pd

In [53]:
auth = OAuth('f716b10140c04e9997c89e1d094745f6', 'b9aa1c134fe244ebaed7b3cba541f6a7')
auth.request_client_credentials()

In [54]:
spotifyClient = Client(auth)

In [55]:
spotifyAPI = spotifyClient.api

In [24]:
def getTrackAttributes(trackIdList = []):
    trackJson = spotifyAPI.tracks_audio_features(trackIdList)
    trackList = trackJson['audio_features']
    trackList = list(filter(None, trackList))
    return trackList

In [44]:
def getAttributeDF(trackFeatList):
    trackDF = pd.DataFrame(trackFeatList)
    trackDF = trackDF.drop(['analysis_url', 'track_href', 'type', 'uri'], axis=1)
    return trackDF

In [63]:
def getPlaylistTrack(play_id,user_id):
    tracks = spotifyAPI.user_playlist_tracks(playlist_id= play_id, user_id= user_id,fields='items(track.id,track.album.name,track.name,track.artists)')
    return tracks


In [65]:
def trackJSONtoDF(tracks,genre):
    flatData = []

    for i in range(len(tracks['items'])):
        trackObj = {}
        trackObj['id'] = tracks['items'][i]['track']['id']
        trackObj['album'] = tracks['items'][i]['track']['album']['name']
        trackObj['track_name'] = tracks['items'][i]['track']['name']
        trackObj['artist'] = tracks['items'][i]['track']['artists'][0]['name']
        trackObj['genre'] = genre
        flatData.append(trackObj)
    return pd.DataFrame(flatData)

In [75]:
def pipeLineFunc(play_id,user_id,genre):
    # Fetch playlist tracks
    plTracks = getPlaylistTrack(play_id,user_id)
    
    # Convert to DF
    plTracksDF =  trackJSONtoDF(plTracks,genre)
    
    # Get list of track ids
    trackIds = list(plTracksDF['id'])
    
    # Get tracks features
    trackFeat = getTrackAttributes(trackIds)
    
    # Convert JSON to DF
    trackFeatDF = getAttributeDF(trackFeat)
    
    # Merging 2 data frames
    resultDF = pd.concat([plTracksDF, trackFeatDF], axis=1, join='inner')
    
    # Droping duplicate ID columns
    resultDF = resultDF.T.drop_duplicates().T

    return resultDF
    
    

In [84]:
res = pipeLineFunc('0VEri7WKnnwZ9Eu7RIhzPK','exsorz','metal')


,album,artist,genre,id,track_name,acousticness,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,American Tragedy (Deluxe Edition),Hollywood Undead,metal,0eUOrB1uVJKQEqZ5EX7pvp,Hear Me Now,0.000200,0.457,214267,0.838,0eUOrB1uVJKQEqZ5EX7pvp,0.000102,7,0.8880,-3.794,1,0.0348,85.989,4,0.359
1,The Connection,Papa Roach,metal,29c925OyK03yOmgFXezETO,Give Me Back My Life,0.000295,0.260,238720,0.860,29c925OyK03yOmgFXezETO,0.143000,1,0.1150,-6.902,1,0.0597,169.996,4,0.318
2,The Paramour Sessions (International Version),Papa Roach,metal,5RxCC8ecpIb6W9MVrJNmSF,...To Be Loved,0.004730,0.478,182120,0.977,5RxCC8ecpIb6W9MVrJNmSF,0.000000,8,0.5090,-3.162,1,0.0751,96.047,4,0.482
3,The Paramour Sessions (International Version),Papa Roach,metal,7v43k9XzWpyNfPVNMyhwUE,Roses On My Grave,0.059900,0.355,193387,0.712,7v43k9XzWpyNfPVNMyhwUE,0.000019,9,0.0817,-6.271,0,0.0360,93.958,4,0.186
4,Getting Away With Murder (Explicit Version),Papa Roach,metal,2NcNJC2jVccC1kKIxEXsZH,Scars,0.087500,0.455,208200,0.929,2NcNJC2jVccC1kKIxEXsZH,0.000000,11,0.2260,-3.295,1,0.0494,89.783,4,0.484


In [90]:
res.T.drop_duplicates().T

,album,artist,genre,id,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,American Tragedy (Deluxe Edition),Hollywood Undead,metal,0eUOrB1uVJKQEqZ5EX7pvp,Hear Me Now,0.0002,0.457,214267,0.838,0.000102,7,0.888,-3.794,1,0.0348,85.989,4,0.359
1,The Connection,Papa Roach,metal,29c925OyK03yOmgFXezETO,Give Me Back My Life,0.000295,0.26,238720,0.86,0.143,1,0.115,-6.902,1,0.0597,169.996,4,0.318
2,The Paramour Sessions (International Version),Papa Roach,metal,5RxCC8ecpIb6W9MVrJNmSF,...To Be Loved,0.00473,0.478,182120,0.977,0,8,0.509,-3.162,1,0.0751,96.047,4,0.482
3,The Paramour Sessions (International Version),Papa Roach,metal,7v43k9XzWpyNfPVNMyhwUE,Roses On My Grave,0.0599,0.355,193387,0.712,1.93e-05,9,0.0817,-6.271,0,0.036,93.958,4,0.186
4,Getting Away With Murder (Explicit Version),Papa Roach,metal,2NcNJC2jVccC1kKIxEXsZH,Scars,0.0875,0.455,208200,0.929,0,11,0.226,-3.295,1,0.0494,89.783,4,0.484
5,Endgame / Appeal To Reason / Siren Song Of The...,Rise Against,metal,2MNLJCavrcJfNdCYk7IHRV,Re-Education (Through Labor),0.000417,0.243,222453,0.947,3.41e-05,11,0.133,-3.388,1,0.102,147.586,4,0.338
6,Endgame / Appeal To Reason / Siren Song Of The...,Rise Against,metal,5uJd3lMR2oQ0e6VDoIvwMR,Savior,0.00154,0.554,242280,0.932,0.000142,5,0.425,-3.424,0,0.0484,112.49,4,0.478
7,Endgame / Appeal To Reason / Siren Song Of The...,Rise Against,metal,7fAURpMGXojfK8Hihfup7k,Tip The Scales,0.00083,0.599,229013,0.971,0.000125,3,0.0827,-3.48,0,0.0816,106.052,4,0.532
8,Endgame / Appeal To Reason / Siren Song Of The...,Rise Against,metal,2KOJuUTky6l8bONSInHn2W,Injection,0.000229,0.42,199213,0.995,8.13e-05,8,0.102,-2.99,0,0.0829,105.605,4,0.427
9,Endgame / Appeal To Reason / Siren Song Of The...,Rise Against,metal,0nRauaGzlpH62t9bAnfv12,Prayer Of The Refugee,0.00131,0.198,199387,0.977,4.62e-05,3,0.23,-3.499,0,0.173,195.795,4,0.351


In [91]:
res

,album,artist,genre,id,track_name,acousticness,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,American Tragedy (Deluxe Edition),Hollywood Undead,metal,0eUOrB1uVJKQEqZ5EX7pvp,Hear Me Now,0.000200,0.457,214267,0.838,0eUOrB1uVJKQEqZ5EX7pvp,0.000102,7,0.8880,-3.794,1,0.0348,85.989,4,0.359
1,The Connection,Papa Roach,metal,29c925OyK03yOmgFXezETO,Give Me Back My Life,0.000295,0.260,238720,0.860,29c925OyK03yOmgFXezETO,0.143000,1,0.1150,-6.902,1,0.0597,169.996,4,0.318
2,The Paramour Sessions (International Version),Papa Roach,metal,5RxCC8ecpIb6W9MVrJNmSF,...To Be Loved,0.004730,0.478,182120,0.977,5RxCC8ecpIb6W9MVrJNmSF,0.000000,8,0.5090,-3.162,1,0.0751,96.047,4,0.482
3,The Paramour Sessions (International Version),Papa Roach,metal,7v43k9XzWpyNfPVNMyhwUE,Roses On My Grave,0.059900,0.355,193387,0.712,7v43k9XzWpyNfPVNMyhwUE,0.000019,9,0.0817,-6.271,0,0.0360,93.958,4,0.186
4,Getting Away With Murder (Explicit Version),Papa Roach,metal,2NcNJC2jVccC1kKIxEXsZH,Scars,0.087500,0.455,208200,0.929,2NcNJC2jVccC1kKIxEXsZH,0.000000,11,0.2260,-3.295,1,0.0494,89.783,4,0.484
5,Endgame / Appeal To Reason / Siren Song Of The...,Rise Against,metal,2MNLJCavrcJfNdCYk7IHRV,Re-Education (Through Labor),0.000417,0.243,222453,0.947,2MNLJCavrcJfNdCYk7IHRV,0.000034,11,0.1330,-3.388,1,0.1020,147.586,4,0.338
6,Endgame / Appeal To Reason / Siren Song Of The...,Rise Against,metal,5uJd3lMR2oQ0e6VDoIvwMR,Savior,0.001540,0.554,242280,0.932,5uJd3lMR2oQ0e6VDoIvwMR,0.000142,5,0.4250,-3.424,0,0.0484,112.490,4,0.478
7,Endgame / Appeal To Reason / Siren Song Of The...,Rise Against,metal,7fAURpMGXojfK8Hihfup7k,Tip The Scales,0.000830,0.599,229013,0.971,7fAURpMGXojfK8Hihfup7k,0.000125,3,0.0827,-3.480,0,0.0816,106.052,4,0.532
8,Endgame / Appeal To Reason / Siren Song Of The...,Rise Against,metal,2KOJuUTky6l8bONSInHn2W,Injection,0.000229,0.420,199213,0.995,2KOJuUTky6l8bONSInHn2W,0.000081,8,0.1020,-2.990,0,0.0829,105.605,4,0.427
9,Endgame / Appeal To Reason / Siren Song Of The...,Rise Against,metal,0nRauaGzlpH62t9bAnfv12,Prayer Of The Refugee,0.001310,0.198,199387,0.977,0nRauaGzlpH62t9bAnfv12,0.000046,3,0.2300,-3.499,0,0.1730,195.795,4,0.351
